In [1]:
import torch
!pip install --upgrade torch-scatter
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster
!pip install --upgrade torch-spline-conv
!pip install torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-1.4.0-cp36-cp36m-linux_x86_64.whl size=3170417 sha256=0b0cd2cf265e9d665a79cdc02203191180b0bb383b09e268a86a0940d742cfbd
  Stored in directory: /root/.cache/pip/wheels/25/00/c4/1637b4b3003f29092f4fe2ad4b40dd10906269c1ac2dc82941
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.3-cp36-cp36m-linux_x86_64.whl size=3966752 sha256=6388cfa9f8baf3ee56a247349a76505b01b66477b53f77693fb7ff34d04c3010
  Stored in directory: /root/.cache/pip/wheels/02/66/2b/befece01c2516f9fb3e7b4d150bb2b871221c73657c9cd7735
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.5-cp36-cp36m-linux_x86_64.whl size=16232288 sha256=1803eecc1351e701a1d0ac02e4c420f61ad1fbcab8d1177333d8dc1e073ff32f
  Stored in directory: /root/.cache/pip/wheels/0a/26/7e/a6d6a80eae5ca39b92bc77773f36cf433d5085de18014382b1
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=tor

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from torch_geometric.datasets import Planetoid

In [4]:
dataset=Planetoid(root='/tmp/cora',name='cora')

Processing...
Done!


In [5]:
len(dataset)

1

In [0]:
#Exploring Dataset

In [7]:
dataset.num_classes

7

In [9]:
dataset.num_edge_features  #no edge embeddings

0

In [10]:
dataset.num_node_features  #node embedd size

1433

In [11]:
dataset.num_features

1433

In [0]:
data=dataset[0]

In [13]:
dataset[1]  #consist of only one graph so error throwing

IndexError: ignored

In [14]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [15]:
data.num_edges

10556

In [0]:
data.num_faces

In [0]:
data.pos  #no geometric data

In [22]:
set(data.y.numpy())

{0, 1, 2, 3, 4, 5, 6}

In [23]:
dataset.num_classes

7

In [0]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [0]:
class net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=GCNConv(dataset.num_node_features,16)
    self.conv2=GCNConv(16,dataset.num_classes)    # each node is classified into 7 labels

  def forward(self,data):
    x,edge_index=data.x,data.edge_index

    x=self.conv1(x,edge_index)
    x=F.relu(x)
    x=F.dropout(x,training=self.training)
    x=self.conv2(x,edge_index)

    return F.log_softmax(x,dim=1)

  

In [0]:
device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [0]:
model=net().to(device)
data=dataset[0].to(device)

In [28]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [0]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=5e-4)

In [38]:
model.train()

net(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [39]:
model.training # tells the self.training as true means tells the model to apply dropout
# during training and not during training as during that time we can set 
# model.training = False by applying model.eval()

True

In [40]:
for epoch in range(200):
  optimizer.zero_grad()
  out=model(data)
  loss=F.nll_loss(out[data.train_mask],data.y[data.train_mask])
  loss.backward()
  optimizer.step()
  print(loss.item())

0.01186368614435196
0.018277721479535103
0.01171889714896679
0.025390880182385445
0.01841690018773079
0.022625837475061417
0.018519053235650063
0.018394475802779198
0.014083122834563255
0.016490967944264412
0.02443462796509266
0.01389167457818985
0.027400793507695198
0.00950644537806511
0.014653367921710014
0.01586567610502243
0.0105260219424963
0.01734302006661892
0.0132233751937747
0.013525734655559063
0.017917074263095856
0.019738921895623207
0.017032794654369354
0.014556118287146091
0.015198959968984127
0.017335861921310425
0.02151113748550415
0.019972337409853935
0.014154907315969467
0.025247521698474884
0.014149696566164494
0.015842903405427933
0.013251051306724548
0.018808012828230858
0.01909003034234047
0.023454466834664345
0.0160923320800066
0.027509363368153572
0.011923606507480145
0.012464829720556736
0.014024816453456879
0.022789938375353813
0.022511055693030357
0.014628883451223373
0.02104392647743225
0.016869544982910156
0.021476343274116516
0.01701674610376358
0.02142865

In [42]:
model.eval()

net(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [43]:
model.training# model.eval() tells the model model is evaluating to remove dropout
# we can set using training=self.training

False

In [47]:
model(data).shape

torch.Size([2708, 7])

In [0]:
# 2708 is total number of nodes 

In [0]:
predval=torch.argmax(model(data),dim=1)

In [50]:
predval

tensor([3, 4, 4,  ..., 0, 3, 3], device='cuda:0')

In [0]:
predmin=torch.argmin(model(data),dim=1)

In [0]:
y1=torch.tensor([1,1,0,0,1])
y2=torch.tensor([0,1,0,1,1])

In [53]:
torch.equal(y1,y2,)

False

In [54]:
data.y

tensor([3, 4, 4,  ..., 3, 3, 3], device='cuda:0')

In [0]:
correctval=predval[data.test_mask].eq(data.y[data.test_mask]).sum().item()

In [0]:
acc=correctval/(data.test_mask.sum().item())

In [59]:
acc*100

80.2

In [0]:
traincount=predval[data.train_mask].eq(data.y[data.train_mask]).sum().item()

In [0]:
trainacc=traincount/(data.train_mask.sum().item())

In [63]:
trainacc*100

100.0